In [2]:
"""
download data without empty ou, when select
['bca_cc_cust_rel.csv',
 'bca_without_mail.csv',
 'dwd_dsc_d365_contract_df.csv',
 'dwd_dsc_d365_opportunity_df.csv',
 'dwd_dsc_finance_estimate_summary_di.csv',
 'dwd_dsc_hr_cost_dtl.csv',
 'dwd_fact_warehouse_billing_detail_dtl.csv'] 
"""
import pandas as pd
import re, os 
import datetime
# import plotly.express as px
# import plotly.graph_objects as go
import numpy as np
import warnings
warnings.filterwarnings("ignore")
# os.listdir('./data_down')



In [89]:
# read data
df_oppa = pd.read_csv('./data_down/' + 'dwd_dsc_d365_opportunity_df.csv', sep = '\001') 
# df_esti = pd.read_csv('./data_down/' + 'dwd_dsc_finance_estimate_summary_di.csv', sep = '\001') 
# df_cost = pd.read_csv('./data_down/' + 'dwd_dsc_hr_cost_dtl.csv', sep = '\001') 
df_bill = pd.read_csv('./data_down/' + 'dwd_fact_warehouse_billing_detail_dtl.csv', sep = '\001') 
# df_bcaw = pd.read_csv('./data_down/' + 'bca_without_mail.csv', sep = '\001') 
rel = pd.read_csv('./data_down/' + 'bca_cc_cust_rel.csv') 
df_bill_type = pd.read_csv('./data_down/' + 'dim_dsc_billing_type_info.csv', sep = '\001') 
"""
pic part data
"""
sap_fina = pd.read_csv('./data_down/' + 'smart_hr_pl_0l.csv', sep = '\001') 
# levin_table = pd.read_excel('./data_down/' + 'OU_Segementation2.xlsx', sheet_name = 2) 
# for i in os.listdir('./data_down'): 
#     print(re.findall( '\w+', i)[0])
#     df = pd.read_csv('./data_down/' + i, sep = '\001') 
def colname_modi(df):
    df.columns = [ re.sub('^\w+.{1}', '', i) for i in list(df.columns)]
for i in [df_oppa, df_bill,  rel,  sap_fina]:
    colname_modi(i)
del i
# rel = rel.drop(['SAP Customer', 'Finance Remark'], axis=1)

rel = rel[~rel['Cost Center'].str.match('中|国')] 
rel.columns = ['cc_in_ou', 'sap_cust', 'bca', 'account_name_en']
rel['five_cc'] = rel['cc_in_ou'].str.slice(0,5)

In [50]:
rel2 = rel[['cc_in_ou', 'sap_cust', 'bca', 'five_cc']].merge(
    rel[['account_name_en', 'five_cc']].query("account_name_en != '(blank)'"
    ), on = 'five_cc', how = 'left').drop_duplicates()
rel2 = rel2.sort_values(['account_name_en', 'bca']).drop_duplicates(subset = 'cc_in_ou')

In [105]:
def oppo_mas(op):
    """
    opportunity select cols.
    """
    def mon_mani(op, col):
        op = op[op[col].astype(str).str.match('(\d+\.)').fillna(False)]
        op[col] = op[col].astype(float) * 1000    
        return op  
    for i in ['contract_value', 'annual_average_revenue', 'annual_average_gross_profit']:
        op = mon_mani(op, i)

    op['actual_close_date'] = op['actual_close_date'].str.slice(0,10).fillna(pd.NaT)
    op['createdon'] = op['createdon'].str.slice(0,10).fillna(pd.NaT)
    op['createdon'] = pd.to_datetime(op['createdon'])
    op['actual_close_date'] = pd.to_datetime(op['actual_close_date'])
    columns = [
        'account_name_en', 'account_name_cn', 'actual_close_date', 'createdon', 'expected_golive_date', 'annual_average_gross_profit',
        'annual_average_revenue', 'bca_ref_number', 'bg', 'commission', 'contract_end_date', 'contract_term',
        'milestone', 'opportunity_type', 'product1', 'product2','product3','product4', 'sector', 'territory',
        'contract_value'
        ]
    op = op[columns]
    return op
    
op = oppo_mas(df_oppa)


In [59]:
"""
my functions.
to customer level. functions set up.
"""
def data_clean(df, substr_year, **kwargs):
    """
    where u can set for month/year level.
    """
    df = df[~df['account_name_en'].isna()]
    df[substr_year] = df[substr_year].astype(str).str.slice(0,6) 
    df = df.groupby(['account_name_en', 'sap_cust', substr_year]).agg({
        **kwargs
    })
    df = df.rename({substr_year: 'month'}, axis = 1)
    return df 

def to_cust_level(df,  substr_year, **kwargs):
    """
    (df,  substr_year, **kwargs) 必须要有ou_code or cost_center!! 
    merge all to cust level \
        to_cust_level(df,  substr_year, **kwargs):
    """
    try:
        df1 = df.merge(rel2, left_on = 'ou_code', right_on = 'cc_in_ou', how  ='left')
    except:
        df1 = df.merge(rel2, left_on = 'cost_center', right_on = 'cc_in_ou', how  ='left')
    return data_clean(df1, substr_year,  **kwargs)

In [60]:
"""
~billing~
bill detail table now having 50+ bms_types. which is unacceptalble/
"""
def data_load_bms():
    df = df_bill.copy()
    df['month'] = df['bms_bill_end_date'].astype(str).str.slice(0,6) 
    df = df.merge(df_bill_type, left_on = 'bms_fee_type', right_on = 'billing_fee_type_code', how = 'left')
    bms = df.pivot_table(index= ['cost_center', 'month'],\
        columns= 'category',
        values='bms_amount',).reset_index().fillna(0)
    return bms
bms = data_load_bms()
bms['yr'] = bms['month'].str.slice(0,4)
bms_cus_yr = to_cust_level(df = bms, substr_year = 'month' ,\
    cost_center = set, 
    人力 = 'sum',
    场地 = 'sum',
    增值 = 'sum',
    报关代理 = 'sum',
    操作 = 'sum',
    短驳运输 = 'sum',
    索赔及返利 = 'sum',
    设备使用 = 'sum' ).reset_index()
bms_cus_yr.head(2)

category,cost_center,month,人力,场地,增值,报关代理,操作,短驳运输,索赔及返利,设备使用,yr
0,ABBOTBJXWS,202003,8153.490566,94013.547170,1368.516509,0.0,18795.386792,0.0,0.0,16845.283019,2020
1,ABBOTBJXWS,202005,16216.742138,182958.566038,1066.343553,0.0,55997.735849,0.0,0.0,17986.415094,2020


In [72]:
"""
~sap~
"""

def load_data_sap():
    df = sap_fina.copy()
    df = df.rename({'prctr': 'cost_center'}, axis =1 )
    return df

sap_fina = load_data_sap()

sap_cust_yr = to_cust_level(sap_fina, 'mont_code', \
    cost_center = set, 
    income_amt = sum,
    gross_profit_amt = sum,
    human_cost_amt = sum, 
    service_outsourcing_amt = sum,
    labor_service_outsourcing_amt = sum
 
    ).reset_index()
    
sap_cust_yr = sap_cust_yr.rename({'yy':'month'}, axis =1 )
sap_cust_yr['lb_cost'] = sap_cust_yr.iloc[:, -3:].sum(axis = 1)
sap_cust_yr.head(2)

,account_name_en,sap_cust,mont_code,cost_center,income_amt,gross_profit_amt,human_cost_amt,service_outsourcing_amt,labor_service_outsourcing_amt,lb_cost
0,"APEX Cargo Service Co., Ltd (ACS)",Freight Forwarding EI,202105,{EIXCNBGGFF},9000982.0,110216.22,672.0,0.0,0.0,672.0
1,"APEX Cargo Service Co., Ltd (ACS)",Freight Forwarding EI,202106,{EIXCNBGGFF},22229261.0,289825.38,0.0,0.0,0.0,0.0


---
finish load

In [110]:
# 删除同一批次导入的数据. 第一次.
op = op[~(op['actual_close_date'] - op['createdon'] < datetime.timedelta(days = 0))] 